In [ ]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch

x = torch.rand(1000, 1)*5
y = 2 * x # + torch.randn(1000, 1) * 0.1

class LCN(nn.Module):
    def __init__(self):
        super(LCN, self).__init__()
        self.hidden1 = nn.Linear(1, 1, bias=False)
        self.hidden2 = nn.Linear(1, 1, bias=False)
        self.hidden3 = nn.Linear(1, 1, bias=False)

    def forward(self, x):
        intermid = []
        x1 = self.hidden1(x)
        x2 = self.hidden2(x1)
        x3 = self.hidden3(x2)
        intermid.append(x+x1)
        intermid.append(x+x1+x2)
        intermid.append(x+x1+x2+x3)
        y = x + x1 + x2 + x3 # x +w1x + w1w2x + w1w2w3x = (1 + w1 + w1w2 + w1w2w3)x
        return y, intermid

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.hidden1 = nn.Linear(1, 1, bias=False)
        self.hidden2 = nn.Linear(1, 1, bias=False)
        self.hidden3 = nn.Linear(1, 1, bias=False)

    def forward(self, x):
        intermid = []
        x1 = self.hidden1(x) + x
        x2 = self.hidden2(x1) + x1
        x3 = self.hidden3(x2) + x2
        intermid.append(x1)
        intermid.append(x2)
        intermid.append(x3)
        y = x3 # x +w1x +w2x + w3x + w1w2x + w1w3x + w2w3x + w1w2w3x = (1 + w1 + w2 + w3 + w1w2 + w1w3 + w2w3 + w1w2w3)x
        return y, intermid

w1_list = [[], []]
num_epochs = 500
for idx in range(2):
  for i in range(10):
    if idx == 0:
      model = LCN()
    else: model = ResNet()
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        outputs, intermid = model(x)
        loss = criterion(outputs, y)
        '''
        print("Intermediate layers' losses")
        for i in range(3):
          print(criterion(intermid[i],y).item())
        print("")
        '''
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        '''
        print("Weights:")
        print(model.hidden1.weight.item())
        print(model.hidden2.weight.item())
        print(model.hidden3.weight.item())
        print("")
        Print progress
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        '''

    w1_list[idx].append(model.hidden1.weight.item())

print("Done.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
sns.histplot(w1_list[0], kde=True, bins=100, color='blue', alpha=0.6)

plt.title('LCN')
plt.xlabel('weight of hidden1')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

plt.figure()
sns.histplot(w1_list[1], kde=True, bins=100, color='orange', alpha=0.6)
plt.title('ResNet')
plt.xlabel('weight of hidden1')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()